In [1]:
#!pip install gensim==3.6.0

In [2]:
import gensim
gensim.__version__

'3.6.0'

In [3]:
import os
os.chdir("/content/drive/MyDrive")

In [4]:
import numpy as np
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from gensim.models import  Word2Vec
from sklearn.metrics import roc_auc_score

In [8]:
dataset = pd.read_csv('SPAM text message 20170820 - Data.csv')
dataset.head()

Category                                            Message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...

In [9]:
################### Replacing Apostrophe to Word #################
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"don\'t", "do not", phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
###################################################################

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
stops = stopwords.words('english')

In [13]:
corpus = []
for i in range(0, dataset.shape[0]):
    review = dataset['Category'][i]
    review = decontracted(review)    
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
#    ps = PorterStemmer()
    review = review.split()
#    review = [word for word in review if not word in set(stops)]
#    review = ' '.join(review)
#    review = [review]
    if review == []:
      review = ['something']
    corpus.append(review)

In [14]:
len(corpus)

5572

In [15]:
corpus[:3]

[['ham'], ['ham'], ['spam']]

## CBOW: sg = 0 Skip-Gram: sg = 1

In [16]:
model_r =  Word2Vec(corpus, min_count=1,sg=1)

In [21]:
means = []
for i in corpus :
    row_means = np.mean(model_r[i],axis=0)
    means.append(row_means)
means = np.array(means) 

X = means
y = np.where(dataset["Category"]=="spam", 1, 0)

<ipython-input-21-cead32b9fd14>:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  row_means = np.mean(model_r[i],axis=0)


In [22]:
X.shape

(5572, 100)

In [23]:
y.shape

(5572,)

In [24]:
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score


X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.3, 
                                                    random_state=2022,
                                                    stratify=y)

from xgboost import XGBClassifier
model_rf = XGBClassifier(random_state=2022)
model_rf.fit( X_train , y_train )

XGBClassifier(random_state=2022)

In [25]:
y_pred_prob = model_rf.predict_proba(X_test)[:,1]

print(roc_auc_score(y_test, y_pred_prob))

1.0
